## Working Test

In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI

load_dotenv()

# Loading The PDF File and Splitting it into Pages
loader = PyPDFLoader("2205.15868v1-CogVideo-Large-scale Pretraining for Text-to-Video.pdf")
pages = loader.load_and_split()

# Chunking the Pages into fixed size chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, length_function=len)
documents = text_splitter.split_documents(pages)

# Converting the documents into embeddings and storing them in a FAISS Vector Store
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vectordb = FAISS.from_documents(documents, embedding=embedding)
store_name = loader.source[:-4]

query = ("What is the main idea of the paper? WHat are the math formulas used in this paper")

docs = vectordb.similarity_search(query=query, k=5)
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
chain = load_qa_chain(llm=llm, chain_type="stuff")
response = chain.run(input_documents=docs, question=query)
print(response)

In [ ]:
# # CHAINS
# https://python.langchain.com/v0.1/docs/modules/chains/

# # DOCUMENT LOADERS
# https://python.langchain.com/v0.2/docs/integrations/document_loaders/


## Testing Class

In [118]:
from langchain_community.document_loaders import (PyPDFLoader, 
                                                  Docx2txtLoader, 
                                                  UnstructuredExcelLoader, 
                                                  UnstructuredPowerPointLoader,
                                                  TextLoader,
                                                  CSVLoader,
                                                  YoutubeLoader)
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI 
import os
import chromadb
files = [
"files/2205.15868v1-CogVideo-Large-scale Pretraining for Text-to-Video.pdf",
'files/IM-Report.docx',
"files/UNIT-2.pptx",
"files/Hperverge OT list final.xlsx",
"files/hey.txt",
"files/test.md",
]
file_path = files[2]
print(file_path)

class Model:
    def __init__(self, llm, embeddings):
        self.llm = llm
        self.embeddings = embeddings


class ScanDocuments:
    def __init__(self):
        pass
    
    def upload_single_file(self, path):
        if path.endswith(".pdf"):
            loader = PyPDFLoader(path)
        elif path.endswith(".docx"):
            loader = Docx2txtLoader(path)
        elif path.endswith(".xlsx"):
            loader = UnstructuredExcelLoader(path, encoding = 'UTF-8')
        elif path.endswith(".pptx"):
            loader = UnstructuredPowerPointLoader(path, encoding = 'UTF-8')
        elif path.endswith(".txt") or path.endswith(".md"):
            loader = TextLoader(path, encoding = 'UTF-8')
        elif path.endswith(".csv"):
            loader = CSVLoader(path, encoding = 'UTF-8')
        data = loader.load_and_split()
        return data
    
    def upload_url(self, url):
        if "www.youtube.com" in url:
            loader =  YoutubeLoader.from_youtube_url(url,add_video_info=True)
        data = loader.load_and_split()
        return data
        
    def upload_zip_file(self):
        pass
    def process_document(self, data):
        doc_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, length_function=len)
        documents = doc_splitter.split_documents(data)
        return documents


class VectorStore:
    def __init__(self):
        self.vectordb = None

    def create_vector_store(self, embedding):
        if os.path.exists("./chroma_vectordb"):
            vectordb = Chroma(persist_directory="./chroma_vectordb", embedding_function=embedding)
        else:
            vectordb = Chroma(collection_name="test",
                              embedding_function=embedding,
                              persist_directory="./chroma_vectordb")
        self.vectordb = vectordb
        return self.vectordb

    def add_documents(self, documents):
        self.vectordb.add_documents(documents)

    def as_retriever(self):
        retriever = self.vectordb.as_retriever()
        return retriever
# ------------------------------------------------------------------------------
# ------------------------------------------------------------------------------
# Google Drive: [https://python.langchain.com/v0.2/docs/integrations/document_loaders/google_drive]
# BibTeX: [https://python.langchain.com/v0.2/docs/integrations/document_loaders/bibtex]
# ArxivLoader, []


files/UNIT-2.pptx

In [134]:
class Search:
    def __init__(self, Model):
        self.model = Model
        self.wikipeadia = None
        self.duckduckgo = None
        self.jinai = None
        self.googleserper = None
        # self.bing = None
        # self.brave = None
        self.base_engine = self.duckduckgo

    def search_base_engine(self, query):
        results = self.base_engine.search(query)
        return results

    def search_multi_engines(self, query):
        wikipedia_results = self.wikipeadia.run(query)
        duckduckgo_results = self.duckduckgo.run(query)
        jina_ai_results = self.jinai.run(query)
        google_serper_results = self.googleserper.run(query)
        
        multi_engine_results = {
            "wikipedia": wikipedia_results,
            "duckduckgo": duckduckgo_results,
            "jina_ai": jina_ai_results,
            "google_serper": google_serper_results
        }
        return multi_engine_results

    def search_best_sites_for_query(self, query):
        prompt = f"""You are a helpful Assistant, You would be Provided a Query by the user, You Have to take the 
        query, extract the main idea behind the query and provide the a sentence which would say:

        Best Sites to Inquire more about query

        For Example:

        User: What is the temperature in Japan?
        Answer: sites to find about weather conditions of countries?:

        Now Create the Reverse Query for the given Query by the user:
        User:{query}
        Answer:
        """
        search_query = self.model.llm.invoke(prompt)
        sites = "https://s.jina.ai/"+search_query
        return sites        


In [123]:
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
model = Model(llm, embedding)

In [136]:
searcher = Search(model)
response = searcher.search_best_sites_for_query("I want to cook indian food specifically Chicken Tikka Masal").content
response

'Answer: sites to find indian recipes specifically chicken tikka masala?: \n'

In [131]:
response.split("Answer:")[-1]

' The best sites to inquire about stylish and spacious bags for your needs are: \n'

## Sample test

In [ ]:
url = "https://www.youtube.com/watch?v=0AW6tWTRLeU"
uploader = ScanDocuments()
data = uploader.upload_url(url)
documents = uploader.process_document(data)

# -----------------------------------------------------------------------
# -----------------------------------------------------------------------
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
model = Model(llm, embedding)

# -----------------------------------------------------------------------
# -----------------------------------------------------------------------
vector_store = VectorStore().create_vector_store(model.embeddings)
vector_store.add_documents(documents)
retriever = vector_store.as_retriever()


In [ ]:
query = "What does Shayne say about Garlic Naan"

qa_chain = RetrievalQA.from_chain_type(
    llm = model.llm,
    chain_type = "stuff",
    retriever = retriever,
    return_source_documents = True,
)
qa_chain.invoke(query)

In [ ]:
from langchain_core.documents import Document

base_description_doc = documents[0]

title = base_description_doc.metadata.get("title")
description = base_description_doc.metadata.get("description")
author = base_description_doc.metadata.get("author")
date = base_description_doc.metadata.get("date")
view_count = base_description_doc.metadata.get("view_count")

summary_doc = Document(
    metadata = base_description_doc.metadata,
    page_content = f"""This is a Youtube video Titled: {title}.
    This video was created by the Channel {author} on {date}. 
    The video has {view_count} views. 
    The Description of the video is: {description}""",
)
summary_doc

new_documents = [summary_doc] + documents
new_documents

## Search

### Jina-ai

In [ ]:
import requests
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
# -----------------------------------------------------------------------
# -----------------------------------------------------------------------
jina_search_url = "https://s.jina.ai/"
search_query = "Tell me About Stable Diffusion CEO and Founder"
response = requests.get(jina_search_url+search_query)

uploader = ScanDocuments()  
text = Document(metadata={'source': '0AW6tWTRLeU'},page_content=str(response.text))
documents = uploader.process_document([text])
documents
# -----------------------------------------------------------------------
# -----------------------------------------------------------------------
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
model = Model(llm, embedding)

# -----------------------------------------------------------------------
# -----------------------------------------------------------------------
# vectordb = FAISS.from_documents(documents, embedding=model.embeddings)
vectordb.add_documents(documents)
# vector_store = VectorStore().create_vector_store()
# vector_store.add_documents(documents)
retriever = vectordb.as_retriever()
# -----------------------------------------------------------------------
# -----------------------------------------------------------------------
qa_chain = RetrievalQA.from_chain_type(
    llm = model.llm,
    chain_type = "stuff",
    retriever = retriever,
    return_source_documents = True,
)
qa_chain.invoke(search_query)


### DuckDuckGo

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun(max_results=5)
query = "Tell me About Stable Diffusion CEO and Founder, Who is the new CEO of Stable Diffusion"
results = search.invoke(query)
# -----------------------------------------------------------------------
# -----------------------------------------------------------------------
uploader = ScanDocuments()  
text = Document(metadata={'source': '0AW6tWTRLeU'},page_content=str(results))
documents = uploader.process_document([text])
documents
# -----------------------------------------------------------------------
# -----------------------------------------------------------------------
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
model = Model(llm, embedding)
# -----------------------------------------------------------------------
# -----------------------------------------------------------------------
vectordb.add_documents(documents)
retriever = vectordb.as_retriever()
# -----------------------------------------------------------------------
# -----------------------------------------------------------------------
qa_chain = RetrievalQA.from_chain_type(
    llm = model.llm,
    chain_type = "stuff",
    retriever = retriever,
    return_source_documents = True,
)
qa_chain.invoke(query)


### Wikipedia

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
results = wikipedia.run("Tell me About Stable Diffusion CEO and Founder")
results
# -----------------------------------------------------------------------
# -----------------------------------------------------------------------
uploader = ScanDocuments()  
text = Document(metadata={'source': '0AW6tWTRLeU'},page_content=str(results))
documents = uploader.process_document([text])
documents
# -----------------------------------------------------------------------
# -----------------------------------------------------------------------
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
model = Model(llm, embedding)
# -----------------------------------------------------------------------
# -----------------------------------------------------------------------
vectordb.add_documents(documents)
retriever = vectordb.as_retriever()
# -----------------------------------------------------------------------
# -----------------------------------------------------------------------
query = "Tell me About Stable Diffusion CEO and Founder"
qa_chain = RetrievalQA.from_chain_type(
    llm = model.llm,
    chain_type = "stuff",
    retriever = retriever,
    return_source_documents = True,
)
qa_chain.invoke(query)

### Serper Search

In [ ]:
from langchain_community.utilities import GoogleSerperAPIWrapper
from dotenv import load_dotenv

load_dotenv() 

search = GoogleSerperAPIWrapper()
search.run("What are the Trending News in Paris Today")


### Brave

In [ ]:
from langchain_community.tools import BraveSearch
tool = BraveSearch.from_api_key(search_kwargs={"count": 3})

### SearchXGN

In [ ]:
from langchain_community.utilities import SearxSearchWrapper
searx = SearxSearchWrapper(searx_host="http://localhost:8888",
                                        unsecure=True)
searx.run("What are the Trending News in Paris Today")

In [ ]:
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI 

# documents =
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


vectordb = Chroma.from_documents(documents, 
                                 embedding=embedding,
                                 persist_directory="chroma"
                                 )

retriever = vectordb.as_retriever(
    search_type = "similarity",
    search_kwargs = {'k':3})

qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = vectordb.as_retriever(),
    # return_source_documents = True,
)

query = "What is video about?"
docs = retriever.get_relevant_documents(query)
qa_chain.invoke(query, documents = docs)

In [ ]:
# -----------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------
# from langchain.vectorstores import Chroma
# from langchain.chains import RetrievalQA

# vectordb  = FAISS.from_documents(documents, embedding=embedding)
# qa_chain = RetrievalQA.from_chain_type(
#     llm = llm,
#     chain_type = "stuff",
#     retriever = vectordb.as_retriever(),
# )

# query = "What is the main idea of the paper? WHat are the math formulas used in this paper"	
# results = qa_chain.invoke(query)
# vectordb.save_local("vectordb")


# vectordb.delete_collection()
# qa_chain.combine_documents_chain_.llm_chain.prompt.messages[0].prompt.template

In [ ]:
from langchain_community.document_loaders import YoutubeLoader

youtube_url = "https://www.youtube.com/watch?v=QsYGlZkevEg"
loader = YoutubeLoader.from_youtube_url(youtube_url,add_video_info=True)
# transcript = loader.load()
# transcript


In [ ]:
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

## Parallel Chains

In [73]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper, WikipediaAPIWrapper, GoogleSerperAPIWrapper
from langchain_core.runnables import RunnableParallel, RunnableMap
from dotenv import load_dotenv
from rich import print
# from langchain_openai import ChatOpenAI  # Assuming you want to use this for summarization

load_dotenv() 

# Initialize search wrappers for each search engine
duckduckgo_search = DuckDuckGoSearchAPIWrapper()
wikipedia_search = WikipediaAPIWrapper()
serperapi_search = GoogleSerperAPIWrapper()

# Create a parallel runnable for the searches
search_chain = RunnableParallel(
    duckduckgo=lambda query: duckduckgo_search.run(query),
    wikipedia=lambda query: wikipedia_search.run(query),
    serperapi=lambda query: serperapi_search.run(query),
)

# Initialize the model for summarization
summarization_model = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
# ChatOpenAI(model="gpt-3.5-turbo")  # Adjust as needed

# Define a function to summarize the results
def summarize_results(results):
    prompt = f"""
    You have received the following search results:

    DuckDuckGo: {results['duckduckgo']}
    Wikipedia: {results['wikipedia']}
    Google Serper: {results['serperapi']}

    Please summarize the most accurate information from these results.
    """
    summary = summarization_model.run(prompt)
    return summary

# Combine the search and summarization into a single chain
combined_chain = RunnableMap(
    search_results=search_chain,
    summary=summarize_results
)

# Run the combined chain with a query
query = "What is the capital of France?"
try:
    results = combined_chain.invoke({"query": query})

    # Print the results from each search engine and the summary
    print("DuckDuckGo Results:", results["search_results"]["duckduckgo"])
    print("Wikipedia Results:", results["search_results"]["wikipedia"])
    print("Google Serper Results:", results["search_results"]["serperapi"])
    print("Summary of Most Accurate Information:", results["summary"])
except Exception as e:
    print("An error occurred during the search or summarization:", str(e))

An error occurred during the search or summarization: https://links.duckduckgo.com/d.js TypeError: argument 'params': 'dict' object cannot be converted to 'PyString'


In [90]:
from rich import print, Pretty

text = "hello"
print(text)

ImportError: cannot import name 'Pretty' from 'rich' (c:\Users\Nilay Kumar\Desktop\langchain_app\venv\Lib\site-packages\rich\__init__.py)

In [112]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper, WikipediaAPIWrapper, GoogleSerperAPIWrapper
from langchain_core.runnables import RunnableParallel, RunnableMap
from dotenv import load_dotenv
# from langchain_openai import ChatOpenAI  # Assuming you want to use this for summarization
from rich import print
load_dotenv() 

# Initialize search wrappers for each search engine
duckduckgo_search = DuckDuckGoSearchAPIWrapper()
wikipedia_search = WikipediaAPIWrapper()
serperapi_search = GoogleSerperAPIWrapper()

# Create a parallel runnable for the searches
search_chain = RunnableParallel(
    duckduckgo=duckduckgo_search.run,
    wikipedia=wikipedia_search.run,
    serperapi=serperapi_search.run,
)

# Initialize the model for summarization
summarization_model = ChatGoogleGenerativeAI(model="gemini-1.5-pro")

# Define a function to summarize the results
def summarize_results(results, query):
    print(results)
    prompt = f"""
    You have received the following search results for the query: {query}

    DuckDuckGo: {results['duckduckgo']}
    Wikipedia: {results['wikipedia']}
    Google Serper: {results['serperapi']}

    Your Job is to provide only the most accurate information from these results for the given query.
    """
    summary = summarization_model.invoke(prompt)
    return summary

query = "Giive me the Recipe for Chicken Tikka Masala and also the steps by step instructions"
first_chain_result = search_chain.invoke(query)
second_chain_result = summarize_results(first_chain_result, query)
print(second_chain_result)

{
    'duckduckgo': 'While the tikka masala cooks, grill your chicken tikka. Thread the chicken on to the skewers and
place them on a prepared tray. Place the tray in the middle rack, in a preheated oven at 240 C or 460 F for 9 to 10
mins. 15. Later turn them with the help of a tong and bake for another 9 to 10 mins. Now cover the lid, cook on 
medium-low for 15-20 minutes, stir occasionally, Cook until it reaches that consistency as shown on video. Next 
take out the whole spices. Add the Chicken Tikka and mix well. Now add ¼ water, and cover the lid let it cook on 
low for 4-5 minutes. Add honey and heavy whipping cream and mix well. Melt butter in a large skillet or dutch oven 
over medium heat. Saute garlic for 1 minute. Season with cumin, paprika, garam masala, chili powder, and salt. Stir
in tomato sauce and cream. Simmer on medium low heat until sauce thickens, about 20 minutes. Add grilled chicken to
the sauce, and simmer for 5 more minutes. Marinate the chicken: In a large bowl, mix the yogurt, lemon juice, oil, 
ginger, garlic, chili powder, cumin, garam masala, turmeric, and cayenne pepper until smooth. Pat the chicken 
pieces dry, then add them to the bowl and mix until they are very well coated. Cover and chill for at least 3 
hours, or overnight. Cover the bowl and leave it in the refrigerator for at least six hours. The longer, the better
for the spices to develop in the meat. When ready to cook, put the meat on skewers and place them on a baking tray.
Cook the chicken tikka in a grilled oven, ordinary oven, air fryer, or instant pot at 350°F/ 180°C.',
    'wikipedia': 'No good Wikipedia Search Result was found',
    'serperapi': 'Chicken tikka masala is a popular curried dish made with boneless chicken, ground spices, onions,
tomatoes, cream and herbs. Instructions · In a bowl, combine chicken with all of the ingredients for the chicken 
marinade; let marinate for 10 minutes to an hour (or ... Missing: steps | Show results with:steps. Slow Cooker 
Chicken Tikka Masala: Same ingredients and instructions for steps 1 and 2. At step 3, add chicken, masala paste, 
and tomato puree to the slow cooker ... Ingredients · 1 cup basmati rice · 1 ½ pounds boneless skinless chicken 
thighs, cut into 1-inch chunks · Kosher salt and freshly ground black ... Missing: steps | Show results with:steps.
Ingredients · 1 cup yogurt · 1 tablespoon lemon juice · 4 teaspoons ground cumin, divided · 1 teaspoon ground 
cinnamon · 2 teaspoons cayenne pepper. Missing: steps | Show results with:steps. Ingredients · 3 boneless, skinless
chicken breasts · ½ cup plain yogurt · 2 tablespoons lemon juice · 6 cloves garlic, minced · 1 tablespoon minced 
... Missing: steps instructions. We start by marinating the chicken in a mixture of yogurt and spices. It needs to 
marinade for at least a couple of hours, to really tenderize ... Step-by-Step Instructions ... Begin by chopping 
the chicken breasts into chunks. Feel free to substitute boneless skinless chicken thighs if you ... Missing: steps
| Show results with:steps. A made-from-scratch Chicken Tikka Masala recipe with the signature yoghurt marinated 
chargrilled chicken smothered in an incredible spice ... ... chicken tikka masala then this recipe is the right 
choice for you. Today I will show you step ... Duration: 9:35. Posted: Jul 24, 2020.'
}

AIMessage(
    content='## Chicken Tikka Masala Recipe & Instructions:\n\nWhile a complete, unified recipe is not available in
the provided search results, we can piece together a coherent recipe and instructions by combining the most 
relevant information:\n\n**Ingredients:**\n\n* **For the Chicken Marinade:**\n    * 1 ½ pounds boneless skinless 
chicken thighs (or breasts), cut into 1-inch chunks\n    * 1 cup plain yogurt\n    * 1 tablespoon lemon juice\n    
* 4 teaspoons ground cumin (divided)\n    * 1 teaspoon ground cinnamon\n    * 2 teaspoons cayenne pepper\n    * 6 
cloves garlic, minced\n    * 1 tablespoon minced ginger\n    * Kosher salt and freshly ground black pepper to 
taste\n* **For the Sauce:**\n    * 1 cup basmati rice (for serving)\n    * Butter \n    * 1 large onion, chopped\n 
* Additional garlic cloves \n    * Ground cumin\n    * Paprika\n    * Garam masala\n    * Chili powder\n    * 
Salt\n    * Tomato sauce\n    * Heavy cream\n    * Honey\n\n**Instructions:**\n\n1. **Marinate the Chicken:** In a 
large bowl, combine all the chicken marinade ingredients. Ensure the chicken is well coated. Cover and refrigerate 
for at least 3 hours, preferably overnight.\n2. **Cook the Chicken Tikka:**\n    * **Oven Method:** Thread 
marinated chicken onto skewers. Place on a baking tray and bake in a preheated oven at 460°F (240°C) for 9-10 
minutes per side, or until cooked through.\n    * **Other Methods:** You can also grill, air fry, or use an Instant
Pot to cook the chicken tikka. Adjust cooking times accordingly.\n3. **Prepare the Sauce:** While the chicken 
cooks, melt butter in a large skillet or dutch oven over medium heat. Sauté onions until softened, then add garlic 
and cook for another minute.\n4. **Bloom the Spices:**  Add cumin, paprika, garam masala, chili powder, and salt to
the pan. Stir and cook for a minute to release their flavors.\n5. **Simmer the Sauce:** Stir in tomato sauce and 
cream. Reduce heat to medium-low and simmer for about 20 minutes, stirring occasionally, until the sauce 
thickens.\n6. **Combine and Finish:** Add the cooked chicken tikka to the sauce and simmer for 5 more minutes. Stir
in honey and heavy whipping cream. Adjust seasoning as needed. \n7. **Serve:** Serve the chicken tikka masala hot 
with basmati rice. \n\n\n**Note:** This recipe is a compilation from various sources.  You may need to adjust the 
ingredient quantities and cooking times to your preference. \n',
    response_metadata={
        'prompt_feedback': {'block_reason': 0, 'safety_ratings': []},
        'finish_reason': 'STOP',
        'safety_ratings': [
            {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False},
            {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False},
            {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False},
            {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}
        ]
    },
    id='run-ec92a09e-5c9c-41b7-9191-d1032e078e6e-0',
    usage_metadata={'input_tokens': 842, 'output_tokens': 574, 'total_tokens': 1416}
)

In [ ]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper, WikipediaAPIWrapper, GoogleSerperAPIWrapper
from langchain_core.runnables import RunnableParallel, RunnableMap
from dotenv import load_dotenv
# from langchain_openai import ChatOpenAI  # Assuming you want to use this for summarization
from rich import print
load_dotenv() 

# Initialize search wrappers for each search engine
duckduckgo_search = DuckDuckGoSearchAPIWrapper()
wikipedia_search = WikipediaAPIWrapper()
serperapi_search = GoogleSerperAPIWrapper()

# Create a parallel runnable for the searches
search_chain = RunnableParallel(
    duckduckgo=duckduckgo_search.run,
    wikipedia=wikipedia_search.run,
    serperapi=serperapi_search.run,
)

# Initialize the model for summarization
summarization_model = ChatGoogleGenerativeAI(model="gemini-1.5-pro")

# Define a function to summarize the results
def summarize_results(results, query):
    print(results)
    prompt = f"""
    You have received the following search results for the query: {query}

    DuckDuckGo: {results['duckduckgo']}
    Wikipedia: {results['wikipedia']}
    Google Serper: {results['serperapi']}

    Your Job is to provide only the most accurate information from these results for the given query.
    """
    summary = summarization_model.invoke(prompt)
    return summary

query = "What are the Harmful effects of Smoking"
first_chain_result = search_chain.invoke(query)


second_chain_result = summarize_results(first_chain_result, query)
print(second_chain_result)